In [ ]:
# For debugging 
import sys, os
sys.path.insert(0, os.getcwd())

# Create a simple graph for testing
from bcipy import bcipy

import numpy as np
from random import shuffle

In [ ]:
# create a session and a graph
session = bcipy.Session.create()
trial_graph = bcipy.Graph.create(session)

In [ ]:
# Create random initialization (training) data and labels
training_data = np.random.random((120,12,500))   
labels = np.asarray([0]*60 + [1]*60)


# Create tensors from the data and labels
init_data = bcipy.Tensor.create_from_data(session,training_data.shape,training_data)
init_labels = bcipy.Tensor.create_from_data(session,labels.shape,labels)


In [ ]:
# Create an input tensor with dummy data
input_tensor = bcipy.Tensor.create_from_data(session, (12, 500), np.random.randn(12, 500))

# Create a scalar that will be populated with the classifier label
classifier_label = bcipy.Scalar.create_from_value(session,-1)

# Create intermediate (virtual) tensors for the intermediate steps of the pipeline
intermediate_tensors = [bcipy.Tensor.create_virtual(session), bcipy.Tensor.create_virtual(session)]

In [ ]:
# create filter parameters
order = 4
bandpass = (8,35) # in Hz
fs = 250

# Create a filter object using the parameters 
filter_obj = bcipy.Filter.create_butter(session,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

In [ ]:
# Create a classifier object
classifier = bcipy.Classifier.create_LDA(session)

In [ ]:
# add the processing nodes to the graph using the factory methods
filter_kernel = bcipy.kernels.FilterKernel.add_filter_node(trial_graph,input_tensor,filter_obj,intermediate_tensors[0], axis = 1)

CSP_kernel = bcipy.kernels.CommonSpatialPatternKernel.add_uninitialized_CSP_node(trial_graph, intermediate_tensors[0], intermediate_tensors[1], init_data, init_labels, 2)

LDA_kernel = bcipy.kernels.ClassifierKernel.add_classifier_node(trial_graph, intermediate_tensors[1], classifier, classifier_label, None, None)

In [ ]:
# verify the session (i.e. schedule the nodes) and ensure the inputs and outputs are connected properly
verify = trial_graph.verify()

if verify != bcipy.BcipEnums.SUCCESS:
    print(verify)
    print("Test Failed D=")

In [ ]:
# Since the graph contains nodes that must be initialzed/trained, we must call initialize() before running the graph
start = trial_graph.initialize()

if start != bcipy.BcipEnums.SUCCESS:
    print(start)
    print("Test Failed D=")

In [ ]:
# RUN!
sts = trial_graph.execute()
    
if sts == bcipy.BcipEnums.SUCCESS:
    # print the value of the most recent trial
    print("Trial {}, Predicted label = {}\n".format(1, classifier_label.data))
else:
    print(f"Trial {1} raised error, status code: {sts}")


print("Test Passed =D")